In [ ]:
import pandas as pd
import numpy as np
from gensim.models import FastText
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
!pip install datasets
from datasets import load_dataset

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 23.6 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3


In [ ]:
!pip install underthesea
from underthesea import text_normalize, word_tokenize

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.9/20.9 MB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 52.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 657.8/657.8 kB 43.5 MB/s eta 0:00:00


In [ ]:
dataset = load_dataset("anti-ai/ViSTS", "STS-Sickr")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating test split:   0%|          | 0/9927 [00:00<?, ? examples/s]

In [ ]:
corpus = dataset['test']['sentence1']

In [ ]:
train_data = [word_tokenize(text_normalize(sentence.lower())) for sentence in corpus]

# Word2Vec with Skip-gram

In [ ]:
model = Word2Vec(train_data,
                 vector_size=512,
                 window=5,
                 min_count=1,
                 workers=4,
                 epochs=25,
                 seed=42,
                 sg=1)

In [ ]:
word2vec = model.wv
len(word2vec.index_to_key)

1813

In [ ]:
dataset_sts13 = load_dataset("anti-ai/ViSTS", "STS13")

In [ ]:
sentences = dataset_sts13['test']['sentence1']

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

def preprocess(sentence):
    tokens = word_tokenize(sentence.lower())
    return [token for token in tokens if token.isalpha()]

target_sentences = [
    "Phụ nữ Saudi được phép thi đấu tại Thế vận hội",
    "2 quả tên lửa Gaza tấn công Israel; không có người bị thương",
    "Coty đề nghị 10 tỷ đô cho Avon",
    "Bão Sandy nhắm vào bờ biển phía đông.",
    "2 người chết trong vụ nổ súng bên ngoài tòa nhà Empire State",
    "NATO lên án mạnh mẽ vụ bắn hạ máy bay Thổ Nhĩ Kỳ của Syria",
    "hài cốt của 167 người được tìm thấy trong hang động Mexico",
    "Jordan mở trại tị nạn đầu tiên cho người Syria",
    "Phiên tòa án giết người Gu Kailai kết thúc ở Trung Quốc",
    "SC từ chối kiến nghị xem xét của chính phủ về phán quyết thuế Vodafone"
]

# Tokenize, preprocess and embed dataset_sentences
dataset_embeddings = []
for sentence in sentences:
    tokens = preprocess(sentence)
    sentence_embedding = np.mean([model.wv[token] for token in tokens if token in model.wv], axis=0)
    if np.isnan(sentence_embedding).any():  # Check for NaN values
        continue
    dataset_embeddings.append(sentence_embedding)

for target_sentence in target_sentences:
    target_tokens = preprocess(target_sentence)
    target_embedding = np.mean([model.wv[token] for token in target_tokens if token in model.wv], axis=0)
    similarities = [cosine_similarity([target_embedding], [embedding])[0][0] for embedding in dataset_embeddings]
    top_5_similarities = np.argsort(similarities)[-5:][::-1]
    print("Target sentences:", target_sentence)
    print("The most 5 similarity sentences:")
    for index in top_5_similarities:
        print(sentences[index], "- Score:", similarities[index])
    print("-----")

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


Target sentences: Phụ nữ Saudi được phép thi đấu tại Thế vận hội
The most 5 similarity sentences:
Thủ tướng ở Tehran, yêu cầu Việt Nam có lập trường rõ ràng về Syria - Score: 0.8326854
Strauss-Kahn của Pháp bị buộc tội liên quan đến hoạt động mại dâm - Score: 0.7808371
một thực thể tồn tại tại một vị trí cụ thể, tại một thời điểm cụ thể, như được quan sát bởi một nhà quan sát ngầm. một cụm từ mô tả và / hoặc thời gian thường được tìm thấy. khung này khác với sự tồn tại ở chỗ vị trí được mô tả như một cơ sở mà một nhà quan sát được hình thành như xác nhận sự tồn tại của thực thể. trong một số trường hợp, vị trí có thể được phục hồi từ các cụm từ chủ yếu thể hiện thông tin ngoại vi. - Score: 0.78066576
không phải là một khởi đầu tốt: máy bay của Hollande bị sét đánh trên đường đến Berlin - Score: 0.777146
Tai nạn máy bay Nigeria: Không có người sống sót - Score: 0.7443221
-----
Target sentences: 2 quả tên lửa Gaza tấn công Israel; không có người bị thương
The most 5 similarity sentences:

# Word2Vec with CBOW

In [ ]:
model = Word2Vec(train_data,
                 vector_size=512,
                 window=5,
                 min_count=1,
                 workers=4,
                 epochs=25,
                 seed=42,
                 sg=0)

# Tokenize, preprocess and embed dataset_sentences
dataset_embeddings = []
for sentence in sentences:
    tokens = preprocess(sentence)
    sentence_embedding = np.mean([model.wv[token] for token in tokens if token in model.wv], axis=0)
    if np.isnan(sentence_embedding).any():  # Check for NaN values
        continue
    dataset_embeddings.append(sentence_embedding)

for target_sentence in target_sentences:
    target_tokens = preprocess(target_sentence)
    target_embedding = np.mean([model.wv[token] for token in target_tokens if token in model.wv], axis=0)
    similarities = [cosine_similarity([target_embedding], [embedding])[0][0] for embedding in dataset_embeddings]
    top_5_similarities = np.argsort(similarities)[-5:][::-1]
    print("Target sentences:", target_sentence)
    print("The most 5 similarity sentences:")
    for index in top_5_similarities:
        print(sentences[index], "- Score:", similarities[index])
    print("-----")

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


Target sentences: Phụ nữ Saudi được phép thi đấu tại Thế vận hội
The most 5 similarity sentences:
một dấu hiệu của một cái gì đó, bằng chứng - Score: 0.8320893
Thủ tướng ở Tehran, yêu cầu Việt Nam có lập trường rõ ràng về Syria - Score: 0.82210374
Chính phủ Libya chiếm lại sân bay Tripoli - Score: 0.8057041
đo độ sâu của một cơ thể nước - Score: 0.8040568
hành động đẩy một cái gì đó đi - Score: 0.8017634
-----
Target sentences: 2 quả tên lửa Gaza tấn công Israel; không có người bị thương
The most 5 similarity sentences:
Usain Bolt thắng 200m  phản ứng của Steven Howard - Score: 0.9300555
Pena Nieto thắng cuộc bầu cử tổng thống Mexico: các cuộc thăm dò dư luận - Score: 0.8020062
xác định hoặc chọn ai đó hoặc một cái gì đó - Score: 0.74399483
Một người chết trong cuộc tấn công mới của Israel vào Gaza - Score: 0.7178556
'Million Muppets March' để bảo vệ Obama - Score: 0.6893854
-----
Target sentences: Coty đề nghị 10 tỷ đô cho Avon
The most 5 similarity sentences:
Chủ tịch Knesset: Obama 

# Fasttext with Skip-gram

In [ ]:
model = FastText(train_data,
                 vector_size=512,
                 window=5,
                 min_count=1,
                 workers=4,
                 epochs=10,
                 seed=42,
                 sg=1)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

def preprocess(sentence):
    tokens = word_tokenize(sentence.lower())
    return [token for token in tokens if token.isalpha()]


# Tokenize, preprocess and embed dataset_sentences
dataset_embeddings = []
for sentence in sentences:
    tokens = preprocess(sentence)
    sentence_embedding = np.mean([model.wv[token] for token in tokens if token in model.wv], axis=0)
    if np.isnan(sentence_embedding).any():  # Check for NaN values
        continue
    dataset_embeddings.append(sentence_embedding)

for target_sentence in target_sentences:
    target_tokens = preprocess(target_sentence)
    target_embedding = np.mean([model.wv[token] for token in target_tokens if token in model.wv], axis=0)
    similarities = [cosine_similarity([target_embedding], [embedding])[0][0] for embedding in dataset_embeddings]
    top_5_similarities = np.argsort(similarities)[-5:][::-1]
    print("Target sentences:", target_sentence)
    print("The most 5 similarity sentences:")
    for index in top_5_similarities:
        print(sentences[index], "- Score:", similarities[index])
    print("-----")

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


Target sentences: Phụ nữ Saudi được phép thi đấu tại Thế vận hội
The most 5 similarity sentences:
Thi thể được tìm thấy tại tai nạn máy bay Nga ở Indonesia - Score: 0.9230879
Cuộc tấn công bằng máy bay không người lái của Mỹ giết chết 5 phiến quân ở Pakistan - Score: 0.8864775
hành động nâng một cái gì đó - Score: 0.86993337
Nghi phạm bắn súng ở Colorado được chăm sóc bởi một bác sĩ tâm thần. - Score: 0.868961
'nhiều thi thể' được tìm thấy ở Syria - Score: 0.86807966
-----
Target sentences: 2 quả tên lửa Gaza tấn công Israel; không có người bị thương
The most 5 similarity sentences:
Barack Obama thừa nhận chúng ta có'nhiều vũ khí hạt nhân hơn chúng ta cần' - Score: 0.9692769
Tai nạn máy bay Nigeria: Không có người sống sót - Score: 0.90119123
hành động nhận một đề nghị - Score: 0.8625704
300 nạn nhân tiềm năng của Jimmy Savile: cảnh sát - Score: 0.86100173
Một tên lửa khác đâm vào Eshkol; không có người bị thương - Score: 0.85155535
-----
Target sentences: Coty đề nghị 10 tỷ đô cho Avo

# Fasttext with CBOW

In [ ]:
model = FastText(train_data,
                 vector_size=512,
                 window=5,
                 min_count=1,
                 workers=4,
                 epochs=10,
                 seed=42,
                 sg=0)

# Tokenize, preprocess and embed dataset_sentences
dataset_embeddings = []
for sentence in sentences:
    tokens = preprocess(sentence)
    sentence_embedding = np.mean([model.wv[token] for token in tokens if token in model.wv], axis=0)
    if np.isnan(sentence_embedding).any():  # Check for NaN values
        continue
    dataset_embeddings.append(sentence_embedding)

for target_sentence in target_sentences:
    target_tokens = preprocess(target_sentence)
    target_embedding = np.mean([model.wv[token] for token in target_tokens if token in model.wv], axis=0)
    similarities = [cosine_similarity([target_embedding], [embedding])[0][0] for embedding in dataset_embeddings]
    top_5_similarities = np.argsort(similarities)[-5:][::-1]
    print("Target sentences:", target_sentence)
    print("The most 5 similarity sentences:")
    for index in top_5_similarities:
        print(sentences[index], "- Score:", similarities[index])
    print("-----")

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


Target sentences: Phụ nữ Saudi được phép thi đấu tại Thế vận hội
The most 5 similarity sentences:
HRW: Jordan đẩy lui người tị nạn Palestine khỏi Syria - Score: 0.9484892
Rimsha Masih được trả tiền bảo lãnh. - Score: 0.9480789
đặt vào đất để phát triển - Score: 0.944438
Các giám sát viên Liên Hợp Quốc vẫn chưa đến được địa điểm thảm sát Syria  video - Score: 0.918675
Các giám sát viên Liên Hợp Quốc vẫn chưa đến được địa điểm thảm sát Syria  video - Score: 0.918675
-----
Target sentences: 2 quả tên lửa Gaza tấn công Israel; không có người bị thương
The most 5 similarity sentences:
Barack Obama thừa nhận chúng ta có'nhiều vũ khí hạt nhân hơn chúng ta cần' - Score: 0.9581981
một công ty cung cấp hàng hóa dịch vụ công cộng - Score: 0.8857399
Tổng thống Ecuador Correa nói không có quyết định về tị nạn Assange - Score: 0.88446695
Tai nạn máy bay Nigeria: Không có người sống sót - Score: 0.8839774
không có rào cản cho đầu tư - Score: 0.8772501
-----
Target sentences: Coty đề nghị 10 tỷ đô cho